In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = ChatGroq(
    temperature=0,
    model = "llama-3.3-70b-versatile"
)

response = llm.invoke("the first person to land on the moon was.....")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
# %pip install langchain langchain-community beautifulsoup4


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)


USER_AGENT environment variable not set, consider setting it to identify your requests.






















Nike Careers









































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format.
    
    The JSON should be a SINGLE DICTIONARY with a key "jobs" containing a list of the postings.
    Each posting in the list should have the following keys: `role`, `experience`, `skills` and `description`.
    
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
# convert str to JSON
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content) # ignore 
json_res

{'jobs': [{'role': '12 Months Head Coach (Senior Store Manager), Nike - Brisbane Airport',
   'experience': None,
   'skills': None,
   'description': 'R-73333, Brisbane, Queensland, Australia, Retail Stores'},
  {'role': 'ATHLETE III',
   'experience': None,
   'skills': None,
   'description': 'R-74705, Nei Mongol, Nei Mongol, China, Retail Stores'},
  {'role': 'Account Engagement Specialist, Germany, Austria, & Switzerland (m/f/d)',
   'experience': None,
   'skills': None,
   'description': 'R-75330, Berlin, Berlin, Germany, Sales & Customer Service'},
  {'role': 'Account Engagement Specialist, Germany, Austria, & Switzerland (m/f/d)',
   'experience': None,
   'skills': None,
   'description': 'R-75331, Mainhausen, Hesse, Germany, Sales & Customer Service'},
  {'role': 'Account Engagement Specialist, UK & Ireland',
   'experience': None,
   'skills': None,
   'description': 'R-75163, London, London, United Kingdom, Sales & Customer Service'},
  {'role': 'Account Executive, SEA&I',

In [7]:
type(json_res)

dict

In [8]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorestore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                        metadatas ={"links":row["Links"]},
                        ids = [str(uuid.uuid4())])
        

In [10]:
links = collection.query(query_texts=["Experience in Python", "Experience in React"],n_results=2)
links

{'ids': [['8508de0a-4630-4c69-b624-495cf18e32c1',
   'f5ecdecd-fb55-44f6-91a2-7bec281d26e8'],
  ['28aaeda1-bd2e-41c1-a001-a103b9575c12',
   'dbbac085-fc61-4768-8ff4-9c5b46b62ec2']],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow',
   'Python, Django, MySQL'],
  ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}],
  [{'links': 'https://example.com/react-portfolio'},
   {'links': 'https://example.com/react-native-portfolio'}]],
 'distances': [[0.996884822845459, 1.0578701496124268],
  [0.9630732536315918, 1.0685749053955078]]}

{'role': '12 Months Head Coach (Senior Store Manager), Nike - Brisbane Airport',
 'experience': None,
 'skills': None,
 'description': 'R-73333, Brisbane, Queensland, Australia, Retail Stores'}

In [11]:
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas', [])
links

NameError: name 'job' is not defined